In [1]:
import os

In [2]:
%pwd

'd:\\A_Category\\iNeuron\\End-to-End-Cancer-Classification-using-MLflow-DVC\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\A_Category\\iNeuron\\End-to-End-Cancer-Classification-using-MLflow-DVC'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

        
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Data/train")

        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

from tensorflow.keras.preprocessing import image
import numpy as np

In [13]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
        

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )
     
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )


    def predict(self):
        imagename = "D:\A_Category\iNeuron\End-to-End-Cancer-Classification-using-MLflow-DVC\inputImage.jpg"
        test_image = image.load_img(imagename, target_size = (350,350))
        test_image = image.img_to_array(test_image)
        test_image = np.expand_dims(test_image, axis = 0)
        result = np.argmax(self.model.predict(test_image), axis=1)
        print("the class is ",result)
        print(type(result))

        result = int(result)
        print(self.train_generator.class_indices)
        class_indice = dict((v,k) for k,v in self.train_generator.class_indices.items())

        print("the class is ", class_indice[result])


In [16]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
        
except Exception as e:
    raise e

[2024-03-18 09:52:36,675: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-18 09:52:36,735: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-18 09:52:36,738: INFO: common: created directory at: artifacts]
[2024-03-18 09:52:36,742: INFO: common: created directory at: artifacts\training]
Found 122 images belonging to 4 classes.
Found 491 images belonging to 4 classes.
{'adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib': 0, 'large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa': 1, 'normal': 2, 'squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa': 3}


In [15]:
config = ConfigurationManager()
training_config = config.get_training_config()
training = Training(config=training_config)
training.get_base_model()
training.train_valid_generator()

training.predict()

[2024-04-08 20:15:43,505: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-04-08 20:15:43,511: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-08 20:15:43,513: INFO: common: created directory at: artifacts]
[2024-04-08 20:15:43,516: INFO: common: created directory at: artifacts\training]
Found 122 images belonging to 4 classes.
Found 491 images belonging to 4 classes.


ValueError: in user code:

    File "d:\A_Category\iNeuron\End-to-End-Cancer-Classification-using-MLflow-DVC\venv\lib\site-packages\keras\engine\training.py", line 2169, in predict_function  *
        return step_function(self, iterator)
    File "d:\A_Category\iNeuron\End-to-End-Cancer-Classification-using-MLflow-DVC\venv\lib\site-packages\keras\engine\training.py", line 2155, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "d:\A_Category\iNeuron\End-to-End-Cancer-Classification-using-MLflow-DVC\venv\lib\site-packages\keras\engine\training.py", line 2143, in run_step  **
        outputs = model.predict_step(data)
    File "d:\A_Category\iNeuron\End-to-End-Cancer-Classification-using-MLflow-DVC\venv\lib\site-packages\keras\engine\training.py", line 2111, in predict_step
        return self(x, training=False)
    File "d:\A_Category\iNeuron\End-to-End-Cancer-Classification-using-MLflow-DVC\venv\lib\site-packages\keras\utils\traceback